<a href="https://colab.research.google.com/github/adityapatil4141/new_project/blob/main/speech_recognition_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd drive/

/content/drive


In [4]:
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 13.5 MB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras import layers
import matplotlib.pyplot as plt
from IPython import display
from jiwer import wer


In [7]:
data_url = "https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2"
data_path = keras.utils.get_file("LJSpeech-1.1",data_url,untar=True)

2748572632/2748572632 [==============================] - 42s 0us/step


#code:

In [80]:
data_path

'/root/.keras/datasets/LJSpeech-1.1'

In [81]:
wavs_path = data_path + "/wavs/"
metadata_path = data_path + "/metadata.csv"

In [82]:
df = pd.read_csv(metadata_path,sep='|',header=None,quoting=3)

In [83]:
metadata_path

'/root/.keras/datasets/LJSpeech-1.1/metadata.csv'

In [84]:
df.head()

,0,1,2
0,LJ001-0001,"Printing, in the only sense with which we are ...","Printing, in the only sense with which we are ..."
1,LJ001-0002,in being comparatively modern.,in being comparatively modern.
2,LJ001-0003,For although the Chinese took impressions from...,For although the Chinese took impressions from...
3,LJ001-0004,"produced the block books, which were the immed...","produced the block books, which were the immed..."
4,LJ001-0005,the invention of movable metal letters in the ...,the invention of movable metal letters in the ...


In [85]:
df.rename(columns={0:"file_name",1:"transcription",2:"normalized_transcription"},inplace=True)

In [86]:
df.head()

,file_name,transcription,normalized_transcription
0,LJ001-0001,"Printing, in the only sense with which we are ...","Printing, in the only sense with which we are ..."
1,LJ001-0002,in being comparatively modern.,in being comparatively modern.
2,LJ001-0003,For although the Chinese took impressions from...,For although the Chinese took impressions from...
3,LJ001-0004,"produced the block books, which were the immed...","produced the block books, which were the immed..."
4,LJ001-0005,the invention of movable metal letters in the ...,the invention of movable metal letters in the ...


In [93]:
df.iloc[0]

file_name                                                          LJ001-0001
normalized_transcription    Printing, in the only sense with which we are ...
Name: 0, dtype: object

In [87]:
#df.columns=['file_name','transcription','normalized_transcription']

In [88]:
# column transcription and normalized_transcription are smilar. just normalized column has full abveration (mr = mister)

In [89]:
df=df[['file_name','normalized_transcription']]

In [90]:
# df=df.sample(frac=1).reset_index(drop=True)

In [91]:
df.head(3)

,file_name,normalized_transcription
0,LJ001-0001,"Printing, in the only sense with which we are ..."
1,LJ001-0002,in being comparatively modern.
2,LJ001-0003,For although the Chinese took impressions from...


In [92]:
df.head()

,file_name,normalized_transcription
0,LJ001-0001,"Printing, in the only sense with which we are ..."
1,LJ001-0002,in being comparatively modern.
2,LJ001-0003,For although the Chinese took impressions from...
3,LJ001-0004,"produced the block books, which were the immed..."
4,LJ001-0005,the invention of movable metal letters in the ...
